In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import tensorflow as tf
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import np_utils


/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:

def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)



In [5]:
data = pd.read_csv('cancer.dat',header=None)
data_array = data.values
#remove rows with missing vals
good_rows = np.invert(np.any(data_array=='?',1))
data = data_array[good_rows]
x= data[:,1:10].astype(np.float32)
y=data[:,10].astype(int)
#one hot encoding - have to use for y in neural nets, bc not automatically a classification problem
le = LabelEncoder()
le.fit(y)
y=le.transform(y)
y=np_utils.to_categorical(y)
(x_train,x_test,y_train,y_test) = train_test_split(x,y,test_size=.2)

In [6]:
np.shape(x_train)

(546, 9)

In [7]:
# Parameters
learning_rate = 0.1
num_steps = 500
batch_size = 500
display_step = 1

# Network Parameters
n_hidden_1 = 12 # 1st layer number of neurons
n_hidden_2 = 8 # 2nd layer number of neurons
num_input = 9 # MNIST data input (img shape: 28*28)
num_classes = 2 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [8]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [9]:
# Create model
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    relu_1 = tf.nn.relu(layer_1)
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(relu_1, weights['h2']), biases['b2'])
    relu_2 = tf.nn.sigmoid(layer_2)

    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(relu_2, weights['out']) + biases['out']
    return out_layer

In [10]:

# Construct model
logits = neural_net(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [11]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = next_batch(batch_size,x_train,y_train)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: x_test,
                                      Y: y_test}))

AttributeError: module 'numpy' has no attribute 'arane'

In [12]:
%debug

> <ipython-input-4-1ba3dd38708c>(6)next_batch()
      4     Return a total of `num` random samples and labels.
      5     '''
----> 6     idx = np.arane(0 , len(data))
      7     np.random.shuffle(idx)
      8     idx = idx[:num]

ipdb> idx
*** NameError: name 'idx' is not defined
ipdb> np.arane
*** AttributeError: module 'numpy' has no attribute 'arane'
ipdb> exit
